In [118]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import gmaps
import requests
import time
from config_googlemaps import gkey

In [119]:
data_2015 = "chronicle_data2015.csv"
data_2016 = "chronicle_data2016.csv"
data_2017 = "chronicle_data2017.csv"
data_2018 = "chronicle_data2018.csv"
state_latlong = "statelatlong.csv"
cities_latlong = "1000-largest-us-cities-by-population-with-geographic-coordinates.csv"
data_2015_2018_cities = "cop_gifts_1518_20181011.csv"

In [120]:
#read to files
data_2015_pd = pd.read_csv(data_2015, encoding = "ISO-8859-1")
data_2016_pd = pd.read_csv(data_2016, encoding = "ISO-8859-1")
data_2017_pd = pd.read_csv(data_2017, encoding = "ISO-8859-1")
data_2018_pd = pd.read_csv(data_2018, encoding = "ISO-8859-1")
state_latlon = pd.read_csv(latlong, encoding = "ISO-8859-1")
data_2015_2018_cities = pd.read_csv(data_2015_2018_cities, encoding= "ISO-8859-1")
cities_latlong = pd.read_csv(cities_latlong, encoding = "ISO-8859-1")

In [121]:
frames = [data_2015_pd, data_2016_pd, data_2017_pd, data_2018_pd]

big_df = pd.concat(frames)
big_df.head()


,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000"
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000"
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000"
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000"
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000"


In [122]:
state_counts = big_df["Where Donor Lives"].value_counts()
state_counts.head()

California    428
New York      268
Texas         197
Florida       164
Illinois      131
Name: Where Donor Lives, dtype: int64

In [123]:
recipient_counts = big_df["Recipient Location"].value_counts()
recipient_counts.head()

California      394
New York        295
Texas           183
Florida         170
Pennsylvania    148
Name: Recipient Location, dtype: int64

In [124]:
renamed_df = big_df.rename(columns={"Where Donor Lives":"State"})
renamed_df.head()

,Year,Donor,State,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000"
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000"
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000"
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000"
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000"


In [125]:
#adding Latitude and Longitude generalized for state locations of donors
coords = pd.merge(renamed_df, state_latlon, how="left", on=["State", "State"])
coords

,Year,Donor,State,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value,State_Abbreviation,Latitude,Longitude
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",NY,40.705626,-73.979680
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680
5,2015,Stephen A. Schwarzman,New York,Investments,"Yale U. (New Haven, Conn.)",Connecticut,Colleges and universities,NaN,"$150,000,000",NY,40.705626,-73.979680
6,2015,Donald Sirkin,Washington,Construction,Lighthouse for the Blind and Visually Impaired...,California,Human and social services,Bequest,"$125,000,000",WA,38.899349,-77.014567
7,2015,Samuel Tak Lee,NaN,"Family wealth, Real estate",Massachusetts Institute of Technology (Cambridge),Massachusetts,Colleges and universities,Gift,"$118,000,000",NaN,NaN,NaN
8,2015,David Geffen,California,Entertainment,U. of California at Los Angeles,California,Colleges and universities,NaN,"$100,000,000",CA,37.271875,-119.270415
9,2015,Gordon and Betty Moore,California,Technology,California Institute of Technology (Pasadena),California,Colleges and universities,Pledge Unrestricted,"$100,000,000",CA,37.271875,-119.270415


In [126]:
latlng_df = coords.rename(columns={"Latitude":"Lat", "Longitude":"Lng"})
latlng_df.head()

,Year,Donor,State,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value,State_Abbreviation,Lat,Lng
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",NY,40.705626,-73.979680
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680


In [127]:
latlng_df = latlng_df.dropna(how='any')

In [128]:
latlng_df.dtypes

Year                          int64
Donor                        object
State                        object
Donor's Source of Wealth     object
Recipient                    object
Recipient Location           object
Cause                        object
Gift type                    object
Gift Value                   object
State_Abbreviation           object
Lat                         float64
Lng                         float64
dtype: object

In [129]:
recipient_df = big_df.rename(columns={"Recipient Location":"State"})
recipient_df.head()

,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,State,Cause,Gift type,Gift Value
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000"
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000"
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000"
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000"
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000"


In [130]:
#adding Latitude and Longitude generalized for state locations of recipients
recipient_coords = pd.merge(recipient_df, state_latlon, how="left", on=["State", "State"])
recipient_coords

,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,State,Cause,Gift type,Gift Value,State_Abbreviation,Latitude,Longitude
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",MA,42.062940,-71.718067
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680
5,2015,Stephen A. Schwarzman,New York,Investments,"Yale U. (New Haven, Conn.)",Connecticut,Colleges and universities,NaN,"$150,000,000",CT,41.518784,-72.757507
6,2015,Donald Sirkin,Washington,Construction,Lighthouse for the Blind and Visually Impaired...,California,Human and social services,Bequest,"$125,000,000",CA,37.271875,-119.270415
7,2015,Samuel Tak Lee,NaN,"Family wealth, Real estate",Massachusetts Institute of Technology (Cambridge),Massachusetts,Colleges and universities,Gift,"$118,000,000",MA,42.062940,-71.718067
8,2015,David Geffen,California,Entertainment,U. of California at Los Angeles,California,Colleges and universities,NaN,"$100,000,000",CA,37.271875,-119.270415
9,2015,Gordon and Betty Moore,California,Technology,California Institute of Technology (Pasadena),California,Colleges and universities,Pledge Unrestricted,"$100,000,000",CA,37.271875,-119.270415


In [131]:
recipient_latlng_df = recipient_coords.rename(columns={"Latitude":"Lat", "Longitude":"Lng"})
recipient_latlng_df.head()

,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,State,Cause,Gift type,Gift Value,State_Abbreviation,Lat,Lng
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",MA,42.062940,-71.718067
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680


In [132]:
recipient_latlng_df = latlng_df.dropna(how='any')

In [133]:
recipient_latlng_df.dtypes

Year                          int64
Donor                        object
State                        object
Donor's Source of Wealth     object
Recipient                    object
Recipient Location           object
Cause                        object
Gift type                    object
Gift Value                   object
State_Abbreviation           object
Lat                         float64
Lng                         float64
dtype: object

In [134]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [135]:
# Store 'Lat' and 'Lng' into  locations 
locations = latlng_df[["Lat", "Lng"]].astype(float)

#donation_amount = latlng_df["Gift Value"].astype(object)



In [136]:
# map based on the donors state of residence
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, 
                                dissipating=False, max_intensity=100, point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius =1 
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [137]:
data_2015_2018_cities

,GiftYear,DonorDisplayName_cu,GiftDonors_aCity,GiftDonors_SourceWealth_cu,GiftRecipients_RecipStateFull,GiftRecipients_RecipCity,GiftCategory_Category,GiftType,GiftValue
0,2017,James Connors,Medford,Investments,Massachusetts,Medford,Education,Bequest,3000000
1,2017,Gund family,Cleveland,"Family wealth, Finance, Real estate",Vermont,Burlington,Colleges and universities,Challenge Gift,6000000
2,2017,John and Susan Herma,Menomonee Falls,Retail,Wisconsin,Milwaukee,Health,Gift Matching,8000000
3,2017,Tesh Wickard,Nashville,Education,Indiana,Nashville,Museums and libraries,Bequest,2300000
4,2017,Lauridsen Family Foundation (Nix and Virginia ...,Des Moines,Pharmaceuticals,Iowa,Des Moines,Arts,Gift,1000000
5,2017,Nolop family,NaN,NaN,South Dakota,Vermillion,Health,NaN,5000000
6,2017,Stephen Prone,Stoughton,Family wealth,Massachusetts,Brockton,Education,Pledge,1000000
7,2015,Hazel Ruby McQuain Charitable Trust (Hazel Rub...,Morgantown,Investments,West Virginia,Reedsville,Colleges and universities,Bequest,6700000
8,2017,Moss Foundation (Chad Moss),NaN,Construction,Florida,Miami,Colleges and universities,NaN,10000000
9,2017,Taner Halicioglu,NaN,Technology,California,San Diego,Colleges and universities,NaN,75000000


In [138]:
recipient_cities_df = data_2015_2018_cities.rename(columns={"GiftRecipients_RecipCity":"City"})
recipient_cities_df.head()

,GiftYear,DonorDisplayName_cu,GiftDonors_aCity,GiftDonors_SourceWealth_cu,GiftRecipients_RecipStateFull,City,GiftCategory_Category,GiftType,GiftValue
0,2017,James Connors,Medford,Investments,Massachusetts,Medford,Education,Bequest,3000000
1,2017,Gund family,Cleveland,"Family wealth, Finance, Real estate",Vermont,Burlington,Colleges and universities,Challenge Gift,6000000
2,2017,John and Susan Herma,Menomonee Falls,Retail,Wisconsin,Milwaukee,Health,Gift Matching,8000000
3,2017,Tesh Wickard,Nashville,Education,Indiana,Nashville,Museums and libraries,Bequest,2300000
4,2017,Lauridsen Family Foundation (Nix and Virginia ...,Des Moines,Pharmaceuticals,Iowa,Des Moines,Arts,Gift,1000000


In [139]:
# merging lat, lng values from cities_latlong csv
city_coords = pd.merge(recipient_cities_df, cities_latlong, how="left", on=["City", "City"])
city_coords

,GiftYear,DonorDisplayName_cu,GiftDonors_aCity,GiftDonors_SourceWealth_cu,GiftRecipients_RecipStateFull,City,GiftCategory_Category,GiftType,GiftValue,State,Lat,Lng
0,2017,James Connors,Medford,Investments,Massachusetts,Medford,Education,Bequest,3000000,Oregon,42.326515,-122.875595
1,2017,James Connors,Medford,Investments,Massachusetts,Medford,Education,Bequest,3000000,Massachusetts,42.418430,-71.106164
2,2017,Gund family,Cleveland,"Family wealth, Finance, Real estate",Vermont,Burlington,Colleges and universities,Challenge Gift,6000000,North Carolina,36.095692,-79.437799
3,2017,Gund family,Cleveland,"Family wealth, Finance, Real estate",Vermont,Burlington,Colleges and universities,Challenge Gift,6000000,Vermont,44.475883,-73.212072
4,2017,John and Susan Herma,Menomonee Falls,Retail,Wisconsin,Milwaukee,Health,Gift Matching,8000000,Wisconsin,43.038902,-87.906474
5,2017,Tesh Wickard,Nashville,Education,Indiana,Nashville,Museums and libraries,Bequest,2300000,Tennessee,36.162664,-86.781602
6,2017,Lauridsen Family Foundation (Nix and Virginia ...,Des Moines,Pharmaceuticals,Iowa,Des Moines,Arts,Gift,1000000,Iowa,41.600545,-93.609106
7,2017,Nolop family,NaN,NaN,South Dakota,Vermillion,Health,NaN,5000000,NaN,NaN,NaN
8,2017,Stephen Prone,Stoughton,Family wealth,Massachusetts,Brockton,Education,Pledge,1000000,Massachusetts,42.083434,-71.018379
9,2015,Hazel Ruby McQuain Charitable Trust (Hazel Rub...,Morgantown,Investments,West Virginia,Reedsville,Colleges and universities,Bequest,6700000,NaN,NaN,NaN


In [140]:
city_latlng_df = city_coords.rename(columns={"Latitude":"Lat", "Longitude":"Lng"})
city_latlng_df.head()

,GiftYear,DonorDisplayName_cu,GiftDonors_aCity,GiftDonors_SourceWealth_cu,GiftRecipients_RecipStateFull,City,GiftCategory_Category,GiftType,GiftValue,State,Lat,Lng
0,2017,James Connors,Medford,Investments,Massachusetts,Medford,Education,Bequest,3000000,Oregon,42.326515,-122.875595
1,2017,James Connors,Medford,Investments,Massachusetts,Medford,Education,Bequest,3000000,Massachusetts,42.418430,-71.106164
2,2017,Gund family,Cleveland,"Family wealth, Finance, Real estate",Vermont,Burlington,Colleges and universities,Challenge Gift,6000000,North Carolina,36.095692,-79.437799
3,2017,Gund family,Cleveland,"Family wealth, Finance, Real estate",Vermont,Burlington,Colleges and universities,Challenge Gift,6000000,Vermont,44.475883,-73.212072
4,2017,John and Susan Herma,Menomonee Falls,Retail,Wisconsin,Milwaukee,Health,Gift Matching,8000000,Wisconsin,43.038902,-87.906474


In [141]:
city_latlng_df = city_latlng_df.dropna(how='any')

In [142]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [143]:
city_locations = city_latlng_df[["Lat", "Lng"]].astype(float)


In [144]:
# map based on the recipient cities
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(city_locations, 
                                dissipating=False, max_intensity=100, point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius =1 
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [146]:
# Store 'Lat' and 'Lng' into  locations 
recipient_locations = recipient_latlng_df[["Lat", "Lng"]].astype(float)






In [147]:
#map based on recipient state locations
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(recipient_locations, 
                                dissipating=False, max_intensity=100, point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius =1 
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))